<a href="https://colab.research.google.com/github/alantang1369/Hurrican_Damage_Detector/blob/main/Hurricane_Damage_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
import zipfile

In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/certificate/satellitehurricaneimages.zip'
urllib.request.urlretrieve(url, 'satellitehurricaneimages.zip')
with zipfile.ZipFile('satellitehurricaneimages.zip', 'r') as zip_ref:
  zip_ref.extractall()

In [3]:
IMG_SIZE = 128
BATCH_SIZE = 64

In [4]:
def preprocess(image, label):

    image = tf.cast(image, tf.float32) / 255.0
    return image, label

In [5]:
import tensorflow as tf

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory='train/',
        image_size=  (IMG_SIZE, IMG_SIZE)
        , batch_size=BATCH_SIZE)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        directory='validation/',
        image_size=  (IMG_SIZE, IMG_SIZE)
        , batch_size=BATCH_SIZE)

Found 10000 files belonging to 2 classes.
Found 2000 files belonging to 2 classes.


In [6]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [7]:
# Normalizes train and validation datasets using the
    # preprocess() function.
    # Also makes other calls, as evident from the code, to prepare them for
    # training.
    # Do not batch or resize the images in the dataset here since it's already
    # been done previously.
train_ds = train_ds.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE).prefetch(
        tf.data.experimental.AUTOTUNE)

# train_ds = train_ds.shuffle(buffer_size=1000).batch(batch_size=64).prefetch(buffer_size=tf.data.AUTOTUNE)

val_ds = val_ds.map(
        preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)

# val_ds = val_ds.batch(batch_size=64).prefetch(buffer_size=tf.data.AUTOTUNE)


In [9]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(filters=10,
                         kernel_size=3, # can also be (3, 3)
                         activation="relu",
                         input_shape=(128, 128, 3)), # first layer specifies input shape (height, width, colour channels)
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.MaxPool2D(pool_size=2, # pool_size can also be (2, 2)
                            padding="valid"), # padding can also be 'same'
  tf.keras.layers.Conv2D(10, 3, activation="relu"),
  tf.keras.layers.Conv2D(10, 3, activation="relu"), # activation='relu' == tf.keras.layers.Activations(tf.nn.relu)
  tf.keras.layers.MaxPool2D(2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation="sigmoid") # binary activation output
])

# Compile the model
model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])



model.summary()

model.fit(train_ds,
          epochs=10,
          steps_per_epoch=len(train_ds),
          validation_data=val_ds,
          validation_steps=len(val_ds))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 10)      280       
                                                                 
 conv2d_5 (Conv2D)           (None, 124, 124, 10)      910       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 62, 62, 10)        0         
 g2D)                                                            
                                                                 
 conv2d_6 (Conv2D)           (None, 60, 60, 10)        910       
                                                                 
 conv2d_7 (Conv2D)           (None, 58, 58, 10)        910       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 29, 29, 10)        0         
 g2D)                                                 